In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
from keras import layers, models
from keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop, SGD
from keras.layers import Dense,Conv2D, Dropout,Activation,MaxPooling2D,Flatten


In [3]:
atributos = '/Users/luceromartinezbonilla/Downloads/rec_fac/list_attr_celeba.txt'
atributos_finales = '/Users/luceromartinezbonilla/Downloads/rec_fac/list_attr_celeba_modificado.txt' 
with open(atributos, 'r') as f:
    print("skipping: " + f.readline())
    print("skipping headers: " + f.readline())
    with open(atributos_finales, 'w') as newf:
        for line in f:
            new_line = ' '.join(line.split())
            newf.write(new_line)
            newf.write('\n')
        

In [6]:
#hacemos un csv con los datos de atributos 
atributos = '/Users/luceromartinezbonilla/Downloads/rec_fac/list_attr_celeba_modificado.txt'
df = pd.read_csv(atributos, sep=' ',  header=None)

In [26]:
#visualizamos el dataframe de atributos
df.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
1,000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
2,000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
3,000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
4,000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1


In [8]:
df.replace(to_replace=-1, value=0, inplace=True) #reemplazamos los valores "-1" por 0
#puede ser haya errores de formato, por lo que nos aseguramos reemplazando todo a numeros
df.replace(to_replace='0', value=0, inplace=True) 
df.replace(to_replace='-1', value=0, inplace=True) 
df.replace(to_replace='1', value=1, inplace=True) 

#Se toman todas las filas, y todas las columnas menos la primera y se transforma a una matriz numpy
x = np.asarray(df.iloc[:, 1:]).astype('int64')
print(x)


In [9]:
files = tf.data.Dataset.from_tensor_slices(df[0]) 
attributes = tf.data.Dataset.from_tensor_slices(x) 
data = tf.data.Dataset.zip((files, attributes)) #Se concatena la columna de nombres (files) y la matriz de atributos (x)


In [10]:
path_to_images = '/Users/luceromartinezbonilla/Desktop/redes neuronales/reconocimiento_facial/archive/img_align_celeba/'

def process_file(file_name, attributes):
    #Se cargan las imágenes uniendo la ruta de la carpeta y el nombre de la imagen
    image = tf.io.read_file(path_to_images + file_name)
    #Se descomprime la imagen
    image = tf.image.decode_jpeg(image, channels=3)
    #Se reescala la imagen
    image = tf.image.resize(image, [192, 192])
    #Se normalizan los pixeles
    image /= 255.0
    #Esta función regresa a la imagen en lugar del nombre obtenido de file_name
    #y los atributos de la imagen sin modificar
    return image, attributes

In [11]:
#Se aplica la función process_file al Dataset unido 
labeled_images = data.map(process_file).batch(batch_size)

In [13]:
epochs = 3
batch_size = 40
optimizer = 'rmsprop'
ih, iw = 192, 192

In [14]:
#usamos una separacion de los datos 80-20
num_train = int(len(df)*0.8) #80% de los datos 
num_test =len(df) - num_train #20% de los datos

epochs_steps = num_train // batch_size
test_steps = num_test // batch_size

d_train = imagen_etiquetada.take(num_train)
d_test = imagen_etiquetada.skip(num_train)


In [15]:
model = Sequential()

#capas convolucionales
model.add(Conv2D(40, (3, 3), input_shape= (192,192,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(100, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(150, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#aplanamiento 
model.add(Flatten())

#redes densas
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.2))

#Capa de salida con 40 neuronas
model.add(Dense(40))
model.add(Activation('sigmoid'))

In [16]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['binary_accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 190, 190, 40)      1120      
                                                                 
 activation (Activation)     (None, 190, 190, 40)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 95, 95, 40)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 93, 93, 100)       36100     
                                                                 
 activation_1 (Activation)   (None, 93, 93, 100)       0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 46, 46, 100)      0         
 2D)                                                    

In [23]:
#entrenamos el mode
model.fit(d_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=d_test,
    validation_steps=test_steps)


Epoch 1/3
5065/5065 [==============================] - 10027s 2s/step - loss: 0.2944 - binary_accuracy: 0.8723
Epoch 2/3
5065/5065 [==============================] - 9850s 2s/step - loss: 0.2657 - binary_accuracy: 0.8840
Epoch 3/3
5065/5065 [==============================] - 9975s 2s/step - loss: 0.2579 - binary_accuracy: 0.8876


In [25]:
#model.save('modelo1_rec_facial.h5')